In [1]:
import pandas as pd
import numpy as np
import string
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df_train = pd.read_csv('../data/train_data.csv')
df_test = pd.read_csv('../data/test_data.csv')

df_train.head()

,filename,artist,title,genre,lyrics,instrumental,tokenized,lemmatized
0,country.00041.wav,Johnny Cash,Goin' By The Book,country,\nYou can see it in the movies and the paper a...,False,"['see', 'movies', 'paper', 'tv', 'news', 'some...","['see', 'movie', 'paper', 'tv', 'news', 'someb..."
1,reggae.00038.wav,Burning Spear,Investigation Dub,reggae,I and I old I know\nI and I old I say\nI and I...,False,"['old', 'know', 'old', 'say', 'reconsider', 's...","['old', 'know', 'old', 'say', 'reconsider', 's..."
2,disco.00070.wav,KC and The Sunshine Band,That's The Way (I Like It),disco,"That's the way, aha, aha\nI like it, aha, aha\...",False,"['thats', 'way', 'aha', 'aha', 'like', 'aha', ...","['thats', 'way', 'aha', 'aha', 'like', 'aha', ..."
3,reggae.00088.wav,Marcia Griffiths,Electric Boogie,reggae,\nIt's electric!\n\n\nYou can't see it\nIt's e...,False,"['electric', 'cant', 'see', 'electric', 'got',...","['electric', 'cant', 'see', 'electric', 'get',..."
4,pop.00017.wav,Britney Spears,(I Can't Get No) Satisfaction,pop,\nI can't get no satisfaction\nI can't get no ...,False,"['cant', 'get', 'satisfaction', 'cant', 'get',...","['cant', 'get', 'satisfaction', 'cant', 'get',..."


In [ ]:
!pip install --upgrade spacy

In [ ]:
!python -m spacy download en_core_web_lg

In [4]:
import spacy

nlp = spacy.load('en_core_web_lg')

OSError: [E050] Can't find model 'en_core_web_lg'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class SentenceVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    # Vectorize a single sentence.
    def _transform(self, tokens):
        vec = np.zeros(nlp.vocab.vectors.shape[1])
        for token in tokens:
          vec = np.add(vec, nlp.vocab[token].vector)
        return vec

    def transform(self, X):
        return np.concatenate(
            [self._transform(row.tokenized).reshape(1, -1) for row in X.itertuples()]
            )

### Naive Bayes

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

In [ ]:
from sklearn.preprocessing import MinMaxScaler

pipe = Pipeline([('vectorizer', SentenceVectorizer()), 
                 ('Normalizing',MinMaxScaler()),
                 ('classifier', MultinomialNB())])
pipe.fit(df_train, df_train['gender'])

Pipeline(steps=[('vectorizer', SentenceVectorizer()),
                ('Normalizing', MinMaxScaler()),
                ('classifier', MultinomialNB())])

In [ ]:
pred = pipe.predict(df_test)
print(classification_report(y_true=df_test['gender'], y_pred=pred))

              precision    recall  f1-score   support

      female       0.38      0.00      0.00     64391
        male       0.51      1.00      0.68     67312

    accuracy                           0.51    131703
   macro avg       0.45      0.50      0.34    131703
weighted avg       0.45      0.51      0.35    131703



Bad performance, only 51% accuracy (as good as guessing)
-> this could be because NaiveBayes has assumption that features are independent of each other. So by doing sentence embedding and adding up the features, this is not fulfilled? Or it's the normalization step that changes the data? (necessary because NaiveBayes can't take negative values)

### Save the models to the drive

In [ ]:
import pickle

#model1 = '/content/drive/MyDrive/Colab Notebooks/gender_prediction/models/NB_countVec.sav'
#pickle.dump(pipe, open(model1, 'wb'))

#model2 = '/content/drive/MyDrive/Colab Notebooks/gender_prediction/models/NB_tfIdf.sav'
#pickle.dump(pipe_tfidf, open(model2, 'wb'))

# load the models with pickle.load(open(filename, 'rb'))